<b><u>Basic Introduction to installing packages using Jupyter and reading/writing tables in GCP</u></b>

How to install packages in jupyter notebook?
- write using the statement <b>!pip install <package_name></b> as shown below

In [14]:
!pip install db_dtypes

You should consider upgrading via the 'C:\Users\abhin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Reading a csv file as a dataframe

In [23]:
import pandas as pd

# CHANGE NAME OF FILE HERE WHICH YOU WANT TO SEE IN THE DATAFRAME
df = pd.read_csv('F:\\Abhi\\code\\Vscode\\phyton\\raw_data\\regchanges.csv')

df.head()

,Season,Regulation change,Change influence,Change difference,competition increase
0,1950,Engine specs set at 1500 cc maximum size for e...,NaN,NaN,NaN
1,1951,NaN,NaN,NaN,NaN
2,1952,NaN,NaN,NaN,NaN
3,1953,NaN,NaN,NaN,NaN
4,1954,Engine specs amended to allow 750 cc maximum e...,NaN,NaN,NaN


Loading the above result into the a table in database/dataset

In [17]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd


# Getting the GCP Cloud Permissions to connect Jupyter and python to Cloud database
credentials = service_account.Credentials.from_service_account_file('F:\\Abhi\\code\\Vscode\\phyton\\credentials\\config.json')
client = bigquery.Client('abhinand-playground',credentials)

# CHANGE NAME OF TABLE HERE AFTER RUNNING CODE TO GET DATAFRAME VALUES
table_id = 'formula1_env.regchanges'

#write_disposition = 'WRITE_APPEND' will append data into the table.
#write_disposition = 'WRITE_TRUNCATE' If the table already exists, BigQuery overwrites the table data.
#write_disposition = 'WRITE_EMPTY' If the table already exists and contains data, a ‘duplicate’ error is returned

job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE') 


job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

BadRequest: 400 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/abhinand-playground/jobs?uploadType=multipart: Invalid field name "Season ". Fields must contain only letters, numbers, and underscores, start with a letter or underscore, and be at most 300 characters long.

querying the table so see the result

In [21]:
from google.cloud import bigquery
from google.oauth2 import service_account
import db_dtypes
# Construct a BigQuery client object.

sql = """
select * from formula1_env.weather
"""
df_1 = client.query(sql).to_dataframe()
df_1.head()

,season,race_name,weather,temperature
0,2014,australian,dry,NaN
1,2014,malaysian,dry,33.0
2,2015,austrian,dry clouded 16 c,16.0
3,2015,monaco,dry clear sky 17 c,17.0
4,2015,russian,dry partly clouded 17 c,17.0


To drop the table - modify the statement below with the table name and run

In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct a BigQuery client object.
credentials = service_account.Credentials.from_service_account_file('C:\\coding\\python\\credentials\\config.json')
client = bigquery.Client('abhinand-playground',credentials)

# If the table does not exist, delete_table raises
# google.api_core.exceptions.NotFound unless not_found_ok is True.
table_id = 'formula1_env.weather'
client.delete_table(table_id, not_found_ok=True)  # Make an API request.
print("Deleted table '{}'.".format(table_id))

Deleted table 'formula1_env.weather'.
